In [27]:
import os
import requests
import pandas as pd
import numpy as np
import pycountry
from geopy.geocoders import Nominatim

from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator import WikibaseIntegrator, wbi_login
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_helpers import execute_sparql_query

In [3]:
frs_facilities = pd.read_parquet('/datasets/skybuilds-drive/edji-knows/FRS_FACILITIES.parquet')

In [4]:
dup_name = frs_facilities.duplicated(['FAC_NAME'], keep=False)
frs_facilities['dup_name'] = np.select([dup_name],[True], default=False)


In [24]:
frs_facilities

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,REGISTRY_ID,FAC_COUNTY,FAC_EPA_REGION,LATITUDE_MEASURE,LONGITUDE_MEASURE,dup_name
0,MERITT AUTO BODY,1911 NEW YORK AVENUE NORTH EAST,WASHINGTON,DC,20002,110010689656,DISTRICT OF COLUMBIA,03,38.917278,-76.976129,False
1,DISTRICT YACHT CLUB,1409 WATER STREET SOUTH EAST,WASHINGTON,DC,20003,110010714833,DISTRICT OF COLUMBIA,03,38.87519,-76.98701,False
2,MICROGRAPHIC ENTERPRISES,1 BRANDYWINE GTWY,WILMINGTON,DE,19801,110002513156,NEW CASTLE,03,39.80776,-75.54898,False
3,TRANSPORT SUPPORT INC,100 SICO ROAD,WILMINGTON,DE,19801,110002513183,NEW CASTLE,03,39.715146,-75.529099,False
4,ALLIED AUTOMOTIVE GROUP,314 BAYWEST BLVD,NEW CASTLE,DE,19720,110002513192,NEW CASTLE,03,39.6446,-75.59491,True
...,...,...,...,...,...,...,...,...,...,...,...
2731423,TAYLOR HOME BUILD,2175 S GARRISON CHAPEL RD,BLOOMINGTON,IN,47403,110071364609,None,05,39.145,-86.642,False
2731424,SHORELINE TRANSFER,1829 NEW HARVEST ROAD,GAINESVILLE,GA,30506,110071364610,None,04,34.2382,-83.8196,False
2731425,"GILLSVILLE HWY. PS, FM AND SEWERS PROJECT",3150 HERITAGE GLEN DRIVE,GAINESVILLE,GA,30501,110071364611,None,04,34.273689,-83.761182,False
2731426,CREEKSIDE MEADOWS,442 WHITEHEAD ROAD,SUGAR HILL,GA,30518,110071364855,None,04,34.103148,-84.063054,True


In [13]:
missing_state_lookup = frs_facilities[
    (frs_facilities.FAC_STATE.isnull())
    &
    (frs_facilities.LATITUDE_MEASURE.notnull())
    &
    (frs_facilities.LONGITUDE_MEASURE.notnull())
][["REGISTRY_ID","LATITUDE_MEASURE","LONGITUDE_MEASURE"]].reset_index(drop=True)

In [20]:
geolocator = Nominatim(user_agent="skybristol@gmail.com")

def lat_lon_lookup(lat,lon):
    coords = ",".join([lat,lon])
    location = geolocator.reverse(coords, exactly_one=True)
    if location:
        return location.raw

missing_state_lookup["nominatim"] = missing_state_lookup.apply(
    lambda x: lat_lon_lookup(x.LATITUDE_MEASURE, x.LONGITUDE_MEASURE),
    axis=1
)

In [21]:
missing_state_lookup.iloc[0].nominatim

{'place_id': 160880026,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 224296165,
 'lat': '47.654984400000004',
 'lon': '-122.37399672568858',
 'display_name': 'Nickerson Business Center, 3837, 13th Avenue West, Queen Anne, Seattle, King County, Washington, 98119, United States',
 'address': {'building': 'Nickerson Business Center',
  'house_number': '3837',
  'road': '13th Avenue West',
  'suburb': 'Queen Anne',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'ISO3166-2-lvl4': 'US-WA',
  'postcode': '98119',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.6549036', '47.6551915', '-122.3742394', '-122.3737541']}